In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

import arviz as az
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)

In [2]:
import numpyro.distributions as dist
from hbmep.model import BaseModel


class MixtureModel(BaseModel):
    LINK = "mixture_model"

    def __init__(self, config: Config):
        super(MixtureModel, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def fn(self, x, a, b, v, L, l, H):
        return (
            L
            + jnp.where(
                jnp.less(x, a),
                0.,
                -l + jnp.true_divide(
                    H + l,
                    jnp.power(
                        1
                        + jnp.multiply(
                            -1
                            + jnp.power(
                                jnp.true_divide(H + l, l),
                                v
                            ),
                            jnp.exp(jnp.multiply(-b, x - a))
                        ),
                        jnp.true_divide(1, v)
                    )
                )
            )
        )

    def _model(self, subject, features, intensity, response_obs=None):
        subject, n_subject = subject
        features, n_features = features
        intensity, n_data = intensity

        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)
        n_feature0 = n_features[0]

        with numpyro.plate(site.n_response, self.n_response):
            """ Global Priors """
            a_mean_global_scale = numpyro.sample("a_mean_global_scale", dist.HalfNormal(100))
            a_shape_global_scale = numpyro.sample("a_shape_global_scale", dist.HalfNormal(100))

            b_scale_global_scale = numpyro.sample("b_scale_global_scale", dist.HalfNormal(100))
            v_scale_global_scale = numpyro.sample("v_scale_global_scale", dist.HalfNormal(100))

            L_scale_global_scale = numpyro.sample("L_scale_global_scale", dist.HalfNormal(10))
            l_scale_global_scale = numpyro.sample("l_scale_global_scale", dist.HalfNormal(100))
            H_scale_global_scale = numpyro.sample("H_scale_global_scale", dist.HalfNormal(10))

            g_1_scale_global_scale = numpyro.sample("g_1_scale_global_scale", dist.HalfNormal(100))
            g_2_scale_global_scale = numpyro.sample("g_2_scale_global_scale", dist.HalfNormal(100))

            print("a_mean_global_scale", a_mean_global_scale.shape)

            # p_scale_global_scale = numpyro.sample("p_scale_global_scale", dist.HalfNormal(100))

            with numpyro.plate("n_feature0", n_feature0):
                """ Hyper-priors """
                # a_mean = numpyro.sample("a_mean", dist.HalfNormal(scale=100))
                # a_scale = numpyro.sample("a_scale", dist.HalfNormal(scale=100))

                # a_mean = numpyro.sample("a_mean", dist.HalfNormal(scale=50))
                # a_shape = numpyro.sample("a_shape", dist.HalfNormal(scale=100))

                a_mean_raw = numpyro.sample("a_mean_raw", dist.HalfNormal(scale=1))
                a_mean = numpyro.deterministic("a_mean", jnp.multiply(a_mean_global_scale, a_mean_raw))

                a_shape_raw = numpyro.sample("a_shape_raw", dist.HalfNormal(scale=1))
                a_shape = numpyro.deterministic("a_shape", jnp.multiply(a_shape_global_scale, a_shape_raw))

                b_scale_raw = numpyro.sample("b_scale_raw", dist.HalfNormal(scale=1))
                b_scale = numpyro.deterministic("b_scale", jnp.multiply(b_scale_global_scale, b_scale_raw))

                v_scale_raw = numpyro.sample("v_scale_raw", dist.HalfNormal(scale=1))
                v_scale = numpyro.deterministic("v_scale", jnp.multiply(v_scale_global_scale, v_scale_raw))

                L_scale_raw = numpyro.sample("L_scale_raw", dist.HalfNormal(scale=1))
                L_scale = numpyro.deterministic("L_scale", jnp.multiply(L_scale_global_scale, L_scale_raw))

                l_scale_raw = numpyro.sample("l_scale_raw", dist.HalfNormal(scale=1))
                l_scale = numpyro.deterministic("sigma_l", jnp.multiply(l_scale_global_scale, l_scale_raw))

                H_scale_raw = numpyro.sample("H_scale_raw", dist.HalfNormal(scale=1))
                H_scale = numpyro.deterministic("H_scale", jnp.multiply(H_scale_global_scale, H_scale_raw))

                g_1_scale_raw = numpyro.sample("g_1_scale_raw", dist.HalfNormal(scale=1))
                g_1_scale = numpyro.deterministic("g_1_scale", jnp.multiply(g_1_scale_global_scale, g_1_scale_raw))

                g_2_scale_raw = numpyro.sample("g_2_scale_raw", dist.HalfNormal(scale=1))
                g_2_scale = numpyro.deterministic("g_2_scale", jnp.multiply(g_2_scale_global_scale, g_2_scale_raw))

                # p_scale_raw = numpyro.sample("p_scale_raw", dist.HalfNormal(scale=1))
                # p_scale = numpyro.deterministic("p_scale", p_scale_global_scale * p_scale_raw)
                print("a_mean", a_mean.shape)

                with numpyro.plate(site.n_subject, n_subject):
                    """ Priors """
                    a_raw = numpyro.sample("a_raw", dist.Gamma(concentration=a_shape, rate=1))
                    a = numpyro.deterministic(site.a, jnp.true_divide(jnp.multiply(a_raw, a_mean), a_shape))
                    # a = numpyro.sample(site.a, dist.TruncatedNormal(loc=a_mean, scale=a_scale, low=0))

                    b_raw = numpyro.sample("b_raw", dist.HalfNormal(scale=1))
                    b = numpyro.deterministic(site.b, jnp.multiply(b_scale, b_raw))

                    v_raw = numpyro.sample("v_raw", dist.HalfNormal(scale=1))
                    v = numpyro.deterministic(site.v, jnp.multiply(v_scale, v_raw))

                    L_raw = numpyro.sample("L_raw", dist.HalfNormal(scale=1))
                    L = numpyro.deterministic(site.L, jnp.multiply(L_scale, L_raw))

                    l_raw = numpyro.sample("l_raw", dist.HalfNormal(scale=1))
                    l = numpyro.deterministic("l", jnp.multiply(l_scale, l_raw))

                    H_raw = numpyro.sample("H_raw", dist.HalfNormal(scale=1))
                    H = numpyro.deterministic(site.H, jnp.multiply(H_scale, H_raw))

                    g_1_raw = numpyro.sample("g_1_raw", dist.HalfCauchy(scale=1))
                    g_1 = numpyro.deterministic(site.g_1, jnp.multiply(g_1_scale, g_1_raw))

                    g_2_raw = numpyro.sample("g_2_raw", dist.HalfCauchy(scale=1))
                    g_2 = numpyro.deterministic(site.g_2, jnp.multiply(g_2_scale, g_2_raw))

                    # p_raw = numpyro.sample("p_raw", dist.HalfNormal(scale=1))
                    # p = numpyro.deterministic("p", p_scale * p_raw)
                    print("a", a.shape)

        """ Outlier Distribution """
        outlier_dist_shape = numpyro.sample("outlier_dist_shape", dist.HalfNormal(5))
        outlier_dist_rate = numpyro.sample("outlier_dist_rate", dist.HalfNormal(1))

        """ Mixture """
        if response_obs is not None:
            outlier_prob = numpyro.sample("outlier_prob", dist.Uniform(0., .2))
        else: # Turn off mixture when predicting
            outlier_prob = numpyro.deterministic("outlier_prob", 0.)

        mixing_distribution = dist.Categorical(probs=jnp.array([1 - outlier_prob, outlier_prob]))

        with numpyro.plate(site.n_response, self.n_response):
            with numpyro.plate(site.n_data, n_data):
                """ Model """
                mu = numpyro.deterministic(
                    site.mu,
                    self.fn(
                        x=intensity,
                        a=a[subject, feature0],
                        b=b[subject, feature0],
                        v=v[subject, feature0],
                        L=L[subject, feature0],
                        l=l[subject, feature0],
                        H=H[subject, feature0]
                    )
                )
                beta = numpyro.deterministic(
                    site.beta,
                    g_1[subject, feature0] + jnp.true_divide(g_2[subject, feature0], mu)
                )
                # beta = numpyro.deterministic(
                #     site.beta,
                #     g_1[subject, feature0] + g_2[subject, feature0] * jnp.power(1 / (mu + 1), p[subject, feature0])
                # )
                # beta = numpyro.deterministic(
                #     site.beta,
                #     g_1[subject, feature0]
                # )

                """ Mixture """
                component_distributions = [
                    dist.Gamma(concentration=jnp.multiply(mu, beta), rate=beta),
                    dist.Gamma(concentration=outlier_dist_shape, rate=outlier_dist_rate)
                ]
                # component_distributions = [
                #     dist.Gamma(concentration=beta, rate=jnp.true_divide(beta, mu)),
                #     dist.Gamma(concentration=outlier_dist_shape, rate=outlier_dist_rate)
                # ]
                Mixture = dist.MixtureGeneral(
                    mixing_distribution=mixing_distribution,
                    component_distributions=component_distributions
                )

                """ Observation """
                numpyro.sample(
                    site.obs,
                    Mixture,
                    obs=response_obs
                )


In [3]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/proc-2023-11-13/config.toml"

config = Config(toml_path=toml_path)
config.BUILD_DIR = os.path.join(config.BUILD_DIR, "model-comparison/mixture-model/adm")
config.RESPONSE = ["PKPK_ADM"]

config.MCMC_PARAMS["num_warmup"] = 5000
config.MCMC_PARAMS["num_samples"] = 1000

model = MixtureModel(config=config)


2023-11-15 09:59:18,631 - hbmep.config - INFO - Verifying configuration ...
2023-11-15 09:59:18,631 - hbmep.config - INFO - Success!
2023-11-15 09:59:18,652 - hbmep.model.baseline - INFO - Initialized base_model


In [4]:
src = "/home/vishu/data/hbmep-processed/human/tms/proc-2023-11-15.csv"
df = pd.read_csv(src)

subset = ["SCS04", "SCA07", "SCA01"]
ind = df[model.subject].isin(subset)
df = df[ind].reset_index(drop=True).copy()

# ind = df[model.intensity] > 0
# df = df[ind].reset_index(drop=True).copy()

df, encoder_dict = model.load(df=df)


2023-11-15 09:59:19,023 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/adm
2023-11-15 09:59:19,024 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/adm
2023-11-15 09:59:19,025 - hbmep.dataset.core - INFO - Processing data ...
2023-11-15 09:59:19,027 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [5]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-11-15 09:59:19,151 - hbmep.model.baseline - INFO - Running inference with base_model ...


a_mean_global_scale (1,)
a_mean (2, 1)
a (3, 2, 1)
a_mean_global_scale (1,)
a_mean (2, 1)
a (3, 2, 1)


  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

a_mean_global_scale (1,)
a_mean (2, 1)
a (3, 2, 1)
a_mean_global_scale (1,)
a_mean (2, 1)
a (3, 2, 1)


In [ ]:
prediction_df = model.make_prediction_dataset(df=df)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=posterior_samples)

model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=posterior_predictive)
model.render_predictive_check(df=df, encoder_dict=encoder_dict, prediction_df=prediction_df, posterior_predictive=posterior_predictive)


2023-11-15 09:58:22,812 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec


a_mean_global_scale (1,)
a_mean (1, 1)
a (1, 1, 1)


2023-11-15 09:58:25,064 - hbmep.utils.utils - INFO - func:predict took: 2.25 sec
2023-11-15 09:58:25,064 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-11-15 09:58:25,537 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/adm/recruitment_curves.pdf
2023-11-15 09:58:25,537 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 0.47 sec
2023-11-15 09:58:25,538 - hbmep.model.baseline - INFO - Rendering posterior predictive checks ...
2023-11-15 09:58:25,790 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/paper/tms/proc-2023-11-13/model-comparison/mixture-model/adm/posterior_predictive_check.pdf
2023-11-15 09:58:25,791 - hbmep.utils.utils - INFO - func:predictive_checks_renderer took: 0.25 sec
2023-11-15 09:58:25,791 - hbmep.utils.utils - INFO - func:render_predictive_check took: 0.25 sec


In [7]:
mcmc.print_summary(prob=.95)



                               mean       std    median      2.5%     97.5%     n_eff     r_hat
             H_raw[0,0,0]      0.76      0.59      0.66      0.00      1.83     66.97      1.08
  H_scale_global_scale[0]      7.72      6.27      6.25      0.05     19.41    136.79      1.04
         H_scale_raw[0,0]      0.72      0.60      0.61      0.00      1.94     70.07      1.04
             L_raw[0,0,0]      0.33      0.46      0.14      0.00      1.40    154.75      1.02
  L_scale_global_scale[0]      3.58      4.28      1.89      0.00     12.69    162.37      1.02
         L_scale_raw[0,0]      0.34      0.43      0.14      0.00      1.25     86.49      1.03
   a_mean_global_scale[0]     99.65     56.21     92.87      3.02    202.73     76.52      1.06
          a_mean_raw[0,0]      1.12      0.60      1.02      0.02      2.30     70.85      1.06
             a_raw[0,0,0]     71.51     82.96     43.79      0.04    246.92    166.26      1.02
  a_shape_global_scale[0]     83.82    

In [ ]:
numpyro_data = az.from_numpyro(mcmc)

""" Model evaluation """
logger.info("Evaluating model ...")

score = az.loo(numpyro_data)
logger.info(f"ELPD LOO (Log): {score.elpd_loo:.2f}")

score = az.waic(numpyro_data)
logger.info(f"ELPD WAIC (Log): {score.elpd_waic:.2f}")


2023-10-27 15:51:53,418 - __main__ - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-10-27 15:51:59,497 - __main__ - INFO - ELPD LOO (Log): 9011.30
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities

In [ ]:
import pickle

dest = os.path.join(model.build_dir, "inference.pkl")
with open(dest, "wb") as f:
    pickle.dump((model, mcmc, posterior_samples), f)


In [ ]:
dest = os.path.join(model.build_dir, "numpyro_data.nc")
az.to_netcdf(numpyro_data, dest)


'/home/vishu/repos/hbmep-paper/reports/paper/tms/link-comparison/rectified_logistic/numpyro_data.nc'